In [1]:
# Import libraries
import requests
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import time
import tqdm.notebook as tq
import matplotlib.pyplot as plt

Скачаем исторические показатели индекса S&P50:

In [ ]:
snp500_values = yf.download('^GSPC', interval='3mo', start=start, end=end, progress=False, show_errors=False)\
    .loc[:, ['Adj Close']]\
    .dropna(axis=0)\
    .query("index.dt.month in [4]")

Будем формировать портфель из акций компаний с максимальной операционной маржой

In [2]:
opmed_ticker_list = requests.get(
    "https://github.com/qununc/financials_repo/blob/815b54427126ef7be539681e03040ac1a3945514/data/opmed_ticker_list.json?raw=true"
).json()

In [3]:
opmed_ticker_list[:5]

['A', 'AA', 'AACG', 'AADI', 'AAIC']

In [4]:
filtered_opm_df = pd.read_csv(
    'https://github.com/qununc/financials_repo/blob/815b54427126ef7be539681e03040ac1a3945514/data/filtered_opm_df.csv.gz?raw=true',
    compression='gzip'
).drop(['2009'], axis=1).set_index('ticker')

[(0, 1062),
 (1, 1462),
 (2, 1756),
 (3, 1964),
 (4, 2154),
 (5, 2318),
 (6, 2474),
 (7, 2634),
 (8, 2816),
 (9, 3013),
 (10, 3239),
 (11, 3527),
 (12, 4639)]

In [5]:
filtered_opm_df = filtered_opm_df.drop(['CHMI', 'LGND', 'BPT'], axis=0)
filtered_opm_df = filtered_opm_df.dropna(axis='rows', thresh=(13 - 4))
filtered_opm_df = filtered_opm_df.fillna(0)
filtered_opm_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1961 entries, A to ZYXI
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2021    1961 non-null   float64
 1   2020    1961 non-null   float64
 2   2019    1961 non-null   float64
 3   2018    1961 non-null   float64
 4   2017    1961 non-null   float64
 5   2016    1961 non-null   float64
 6   2015    1961 non-null   float64
 7   2014    1961 non-null   float64
 8   2013    1961 non-null   float64
 9   2012    1961 non-null   float64
 10  2011    1961 non-null   float64
 11  2010    1961 non-null   float64
dtypes: float64(12)
memory usage: 199.2+ KB


In [6]:
filtered_opm_df.head(3)

,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
ticker,,,,,,,,,,,,
A,21.3167,15.8457,18.2258,18.3964,18.0456,14.6359,12.9272,10.3508,9.9127,16.3167,16.1905,10.3968
AAIC,20.3575,22.7822,9.0810,25.0151,42.1978,52.5537,72.2290,71.6889,77.3337,65.0669,68.2234,60.0955
AAL,0.0000,0.0000,6.6968,5.9630,9.9268,12.6053,15.1354,9.9625,5.2313,0.5955,0.0000,1.3893


In [7]:
# filtered_opm_df['2021'][filtered_opm_df['2021'] < 100].sort_values(ascending=False)

In [8]:
for year in filtered_opm_df.columns:
    print(filtered_opm_df[year][filtered_opm_df[year] < 100].sort_values(ascending=False)[:8], '\n********\n')

ticker
SJT     95.6022
SBR     94.9857
EFC     94.5768
HRZN    94.3298
VOC     93.2028
MSB     92.5195
NYMT    92.2984
PBT     90.7768
Name: 2021, dtype: float64 
********

ticker
MSB     95.3793
HRZN    94.0697
BBDC    91.7751
SBR     91.5926
PBT     90.9363
PVL     90.2629
BXMT    84.1227
SJT     83.8197
Name: 2020, dtype: float64 
********

ticker
PVL     98.3778
MSB     97.5950
PBT     94.6831
HRZN    94.1913
SBR     93.9065
VOC     93.3122
ARR     91.5869
BBDC    90.9009
Name: 2019, dtype: float64 
********

ticker
MSB     97.5807
PBT     95.8713
FBRT    95.6884
SBR     95.0529
VOC     93.2860
HRZN    92.4992
SJT     92.3395
MTR     91.6741
Name: 2018, dtype: float64 
********

ticker
PVL     98.1020
MTR     97.0096
FBRT    96.8110
SJT     96.0552
PBT     95.9201
TPL     93.7355
SBR     93.3491
MSB     92.3675
Name: 2017, dtype: float64 
********

ticker
TPL     95.1625
FBRT    92.9818
HRZN    92.8086
PVL     92.0726
PBT     91.7407
SBR     91.4930
MSB     90.5387
MFA     90.0291


In [9]:
vivi_df = filtered_opm_df.T.reset_index()
vivi_df.columns = vivi_df.iloc[0]
vivi_df = vivi_df.iloc[1:]
vivi_df

,2021,21.3167,20.3575,0.0,3.352,12.9562,7.6261,29.7824,26.5731,17.6463,...,32.4879,9.9551,17.3983,11.8118,7.4588,0.0,31.9959,9.7853,11.7466,17.1649
1,2020,15.8457,22.7822,0.0000,8.7603,19.7912,7.4202,24.1473,25.7075,15.4080,...,26.5924,0.0000,14.6358,11.9379,0.0464,0.0000,29.9026,8.2975,14.6599,12.7917
2,2019,18.2258,9.0810,6.6968,0.8694,14.2779,6.9748,24.5720,30.9987,0.0000,...,34.4828,14.2505,15.4292,8.3985,1.0519,7.0230,28.7700,6.2413,14.9008,24.3359
3,2018,18.3964,25.0151,5.9630,0.5745,12.7552,6.3073,26.6940,24.0384,10.5889,...,40.3657,0.4261,14.4618,20.2337,3.3265,6.1404,29.0129,5.2577,14.3876,32.4905
4,2017,18.0456,42.1978,9.9268,3.9086,18.3191,6.0830,26.7604,29.7305,11.2209,...,46.9718,10.2431,8.6513,21.9806,1.8025,7.1640,28.7356,4.7524,11.8657,38.1700
5,2016,14.6359,52.5537,12.6053,3.0624,20.7288,8.2319,27.8354,33.1904,9.1492,...,26.4632,10.7076,2.2384,27.7455,0.5448,10.9688,25.1228,5.7406,9.0602,4.8073
6,2015,12.9272,72.2290,15.1354,4.3010,19.8817,8.4808,30.4773,34.1593,6.7761,...,22.3434,7.7912,1.0137,27.6606,0.0000,12.0356,11.4376,8.8190,9.9356,0.0000
7,2014,10.3508,71.6889,9.9625,3.9147,19.0855,8.6522,28.7223,32.1804,9.7806,...,23.0302,22.1963,5.3262,31.0846,0.0000,3.6571,17.1369,10.0564,13.3904,0.0000
8,2013,9.9127,77.3337,5.2313,7.9241,17.3834,10.1684,28.6695,31.7972,11.2735,...,13.7420,23.1129,15.4143,33.6833,1.5558,2.1005,15.1283,10.2394,12.4981,0.0000
9,2012,16.3167,65.0669,0.5955,5.5168,14.5932,10.5933,35.2960,30.5377,13.7598,...,16.8268,23.4229,16.4983,43.6628,1.3333,3.6549,16.3745,10.8356,12.6170,6.9001
10,2011,16.1905,68.2234,0.0000,4.9978,8.3273,10.7713,31.2151,27.7846,10.8066,...,14.3751,23.0042,18.5092,41.1771,3.5240,2.2648,9.3078,7.8035,12.9679,9.0811


In [10]:
len(opmed_ticker_list)

4639

In [11]:
stocks = pd.read_csv(
    'https://github.com/qununc/financials_repo/blob/815b54427126ef7be539681e03040ac1a3945514/data/stock_final_2012-2022.csv.xz?raw=true',
    compression='xz'
)
stocks[stocks['Name'].isin(filtered_opm_df.index)]
stocks.head()

,Date,Adj Close,Volume,Name
0,2008-04-01,23.166740,239250504.0,A
1,2009-04-01,13.239071,279708765.0,A
2,2010-04-01,18.532087,332727075.0,A
3,2011-04-01,33.316044,308369858.0,A
4,2012-04-01,25.636250,279486062.0,A


In [12]:
stocks.shape

(38562, 4)

In [13]:
stocks.rename({'Adj Close' : 'adj_close'}, axis=1, inplace=True)
stocks['Date'] = pd.to_datetime(stocks['Date'], infer_datetime_format=True)

stocks = stocks.drop_duplicates(ignore_index=True)
stocks_pivoted = stocks.pivot_table(index='Name', columns='Date', values='adj_close', aggfunc='mean')
# stocks_pivoted = stocks_pivoted.dropna(axis=0, thresh=3) # drop rows with  less then 3 points of data
stocks_pivoted.head()

Date,2008-04-01,2009-04-01,2010-04-01,2011-04-01,2012-04-01,2013-04-01,2014-04-01,2015-04-01,2016-04-01,2017-04-01,2018-04-01,2019-04-01,2020-04-01,2021-04-01,2022-04-01
Name,,,,,,,,,,,,,,,
A,23.166740,13.239071,18.532087,33.316044,25.636250,28.232319,38.206318,36.305489,42.085457,57.026402,59.846397,72.932610,87.272156,146.784302,119.083038
AA,74.858009,22.716898,22.342682,35.542171,19.800377,17.939407,34.601208,26.110579,21.987661,32.542919,46.726246,23.333220,11.203135,36.719173,67.800003
AACG,1.044650,0.712225,0.240888,0.802959,0.357990,0.371514,0.406480,0.555307,0.444054,0.435422,0.448928,2.700000,1.260000,3.080000,1.180000
AAIC,7.232046,1.832119,4.629126,8.469669,6.706148,9.596067,11.192897,9.143547,6.932003,8.668079,7.776839,6.146327,2.970000,4.060000,3.050000
AAL,2.356854,2.290862,8.117003,8.399826,12.566744,15.479815,40.500172,37.919956,27.128143,48.734039,37.069473,32.178577,13.070000,21.209999,18.770000


In [14]:
stocks_pivoted.dropna().index

Index(['A', 'AA', 'AACG', 'AAIC', 'AAL', 'AAME', 'AAON', 'AAP', 'AAPL', 'AATC',
       ...
       'YUM', 'YVR', 'ZBH', 'ZBRA', 'ZD', 'ZEUS', 'ZIVO', 'ZNH', 'ZUMZ',
       'ZYXI'],
      dtype='object', name='Name', length=2228)

In [16]:
filtered_opm_df.index

Index(['A', 'AAIC', 'AAL', 'AAME', 'AAON', 'AAP', 'AAPL', 'AAT', 'AAWW', 'ABB',
       ...
       'YUM', 'ZBH', 'ZBRA', 'ZD', 'ZEUS', 'ZNH', 'ZTS', 'ZUMZ', 'ZWS',
       'ZYXI'],
      dtype='object', name='ticker', length=1961)

In [22]:
stocks_pivoted[stocks_pivoted.index.isin(filtered_opm_df.index)]

Date,2008-04-01,2009-04-01,2010-04-01,2011-04-01,2012-04-01,2013-04-01,2014-04-01,2015-04-01,2016-04-01,2017-04-01,2018-04-01,2019-04-01,2020-04-01,2021-04-01,2022-04-01
Name,,,,,,,,,,,,,,,
A,23.166740,13.239071,18.532087,33.316044,25.636250,28.232319,38.206318,36.305489,42.085457,57.026402,59.846397,72.932610,87.272156,146.784302,119.083038
AAIC,7.232046,1.832119,4.629126,8.469669,6.706148,9.596067,11.192897,9.143547,6.932003,8.668079,7.776839,6.146327,2.970000,4.060000,3.050000
AAL,2.356854,2.290862,8.117003,8.399826,12.566744,15.479815,40.500172,37.919956,27.128143,48.734039,37.069473,32.178577,13.070000,21.209999,18.770000
AAME,1.471958,0.676552,1.234251,1.891998,2.557273,3.711583,3.683750,3.481628,3.827819,3.639684,2.633808,2.404644,1.642303,4.273946,3.033242
AAON,4.942863,5.192170,6.186755,8.825144,7.704118,13.759431,21.049248,21.385914,26.361738,35.593182,32.347404,49.268433,53.661415,62.259750,48.740002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZD,20.000000,19.617392,18.991304,24.547827,22.973913,36.965218,44.226086,59.078262,54.930435,73.991302,75.313042,77.295654,54.965218,119.608696,88.360001
ZEUS,69.792053,23.194458,21.843235,26.266115,15.720266,23.561548,23.873524,16.909571,26.647728,19.073830,20.068403,13.474362,11.671588,29.373936,34.330002
ZNH,10.971809,11.626014,17.214485,22.486942,18.345486,18.246649,13.704590,54.440166,26.754959,39.666912,38.554474,33.943115,22.090000,31.070000,27.030001
